## Cleaning data in order to obtain the dummy variables (used for elastic net and neural network models)

In [1]:
import  numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeCV

from sklearn.preprocessing import QuantileTransformer, quantile_transform

from sklearn.metrics import median_absolute_error, r2_score, mean_squared_error
# ....

from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
# import lightgbm as lgb 

from lightgbm import LGBMRegressor,LGBMClassifier
from sklearn.pipeline import Pipeline, make_pipeline

from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime

# tuner.get_best_hyperparameters(num_trials=1)[0].values
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from datetime import datetime



%matplotlib inline

## Load the data

Let's load the data and have a look at it (make sure you copy the folder FFChallenge_v5 in your working directory). 

In [2]:
# background.csv contains 4,242 rows (one per family) and 13,027 columns
#                index by challengeID: A unique numeric identifier for each child.
#                features: 13,026 background variables asked from birth to age 9, 
#                which you may use in building your model.
background = pd.read_csv('FFChallenge_v5/background.csv', sep=',', header=0,index_col=0,low_memory=False)



# train.csv contains 2,121 rows (one per child in the training set) and 7 columns.
train = pd.read_csv('FFChallenge_v5/train.csv', sep=',', header=0, index_col=0,low_memory=False)

#constantVariables.txt gives the column names of variables that are constant in the data.
#We recommend that the first step in any analysis be to remove the variables that are constant.
constantVariables = pd.read_csv('FFChallenge_v5/constantVariables.txt',header=None, index_col=0,low_memory=False)



########### Holdout dataset for internal testing only
test = pd.read_csv('test.csv',header=0, index_col=0,low_memory=False)
leaderboard = pd.read_csv('leaderboard.csv', header=0, index_col=0,low_memory=False)
leaderboard = leaderboard.dropna()

In [3]:
# Function that maps negative integers from -1 to -9 to NAN (since these are non-response codes)
def map_negative_to_nan(cell_number):
    non_answers_numbers = list(range(-1,-10,-1))
    delta = 0.00001
    bool_val = any([True if non_answer_val-delta<cell_number and  non_answer_val+delta>=cell_number else False for non_answer_val in non_answers_numbers])
    if bool_val:
        return np.nan
    else:
        return cell_number


## Cleaning the data

In [4]:
# dataframe with dummy variables when someone doesn't answer a question (will be merged back to the background df in a later cleaning step)
# Drop constant columns
background= background.drop(constantVariables.index.values, axis = 1)

numerical_columns = [c for c,v in background.dtypes.iteritems() if v in [np.float,np.int,np.int64]]
non_numerical_columns = [c for c,v in background.dtypes.iteritems() if v not in [np.float,np.int,np.int64]]

start = datetime.now()

background_copy = background.copy()
# Next drop the columns that have no 1s (that means there were no questions that werent answered, so no dummy vars necessary)

print(datetime.now()-start)   
# It seems like it's much faster to apply the map, then merge everything back together
background_copy_numerical = background_copy[numerical_columns].applymap(lambda x:map_negative_to_nan(x))

# New backgrounddf with non answers switched to "NA"
background = background_copy_numerical.merge(background.copy()[non_numerical_columns],how = 'inner', left_index = True, right_index = True)
print(datetime.now()-start)

# A bunch of new NA values were created, these ones need to be dropped (now that we have the associated dummy variables)
mask = (background.isna().sum(axis=0)/len(background)<0.8)
background = background[mask[mask].index]



0:00:00.074009
0:02:55.135191


In [5]:
# Final stats and useful variables associated with each column
# Finding numerical, continuous and categorical ccolumns
# Columns with more than 20 levels or with non-numerical data are classified as categorical variables

numerical_columns = [c for c,v in background.dtypes.iteritems() if v in [np.float,np.int,np.int64]]
non_numerical_columns = [c for c,v in background.dtypes.iteritems() if v not in [np.float,np.int,np.int64]]
categorical_bools = background.copy().loc[:,(background.apply(pd.Series.nunique) < 20).values]

# The categorical columns are the intersection of both the non_numerical columns and the ones that have less than 15 distinct levels
categorical_cols_lst = list(set(categorical_bools.columns).union(set(non_numerical_columns)))
continuous_cols_lst = list(set(numerical_columns).difference(set(categorical_cols_lst)))

print('Number of categorical columns %s' % len(categorical_cols_lst))

print('Number of continuous columns %s' % len(continuous_cols_lst))

Number of categorical columns 4761
Number of continuous columns 507


### Splitting the data into training and CV set

In [6]:
X_train, X_CV, y_train, y_CV = train_test_split(background.loc[train.index] , train, test_size=0.2 , random_state=42)

## Performing imputation and obtaining the dummy variables

In [7]:
# Get Dummies

from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from datetime import datetime

start = datetime.now()


# background_test = background.copy()
continuous_vars = background[continuous_cols_lst]
categorical_vars = background[categorical_cols_lst]
# background_test = pd.get_dummies(background_test,columns = categorical_cols_lst, prefix=categorical_cols_lst,drop_first = True)

X_train_continuous= continuous_vars.loc[X_train.index.values]
X_train_categorical= categorical_vars.loc[X_train.index.values]



# Impute continuous data
imputer = SimpleImputer(missing_values=np.nan, strategy='mean',copy = False)
# imputer = KNNImputer(n_neighbors=2, weights="uniform")
imputer.fit(X_train_continuous) # Impute on the train set
background_imputed_continuous=pd.DataFrame(imputer.transform(continuous_vars)) # Transform the test set using the impute transform from the TRAINING data
background_imputed_continuous.columns=continuous_vars.columns
background_imputed_continuous.index=continuous_vars.index


# Imputing cagtegorical data using mode
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent',copy = False)
imp_mode.fit(X_train_categorical) # Impute on the train set
background_imputed_categorical=pd.DataFrame(imp_mode.transform(categorical_vars))
background_imputed_categorical.columns=categorical_vars.columns
background_imputed_categorical.index=categorical_vars.index


print(background_imputed_continuous.shape)
# Merge the continuous and categorical variables back together
background_imputed_tot = background_imputed_continuous.merge(background_imputed_categorical, left_index = True,
                                                             right_index=True, how="inner")


print(background_imputed_tot.shape)
# Create dummy variables now that variables have been filled (mode imputation doesn't work if the variables are already dummy variables)
background_imputed_tot = pd.get_dummies(background_imputed_tot,columns = categorical_cols_lst,
                                        prefix=categorical_cols_lst)

print(background_imputed_tot.shape)


# Dropping variables that will not provide any useful information for classification/regression
# This includes dummy variables that only appear 1 time in the entire dataset, or 4141 times (all but 1 time)

lst_to_drop = list()

sums = background_imputed_tot.sum()

for col in background_imputed_tot.columns:
    # Ignore all continuous variables, since they may unintentionally have a sum that is close to 0
    if col not in continuous_cols_lst and (sums.loc[col]<=1 or sums.loc[col]>=len(background_imputed_tot)-1):
        lst_to_drop.append(col)

# testvvvvv = pd.DataFrame(a)
# testvvvvv.sort_values(by=[1])

background_imputed_tot = background_imputed_tot.drop(lst_to_drop, axis = 1)
print(background_imputed_tot.shape)

# Get training values where values never occur
X_train = background_imputed_tot.loc[X_train.index.values]


# Figuring out if the categorical columns do not appear at all in the training set and dropping these columns
lst_to_drop = list()

sums = X_train.sum()

for col in X_train.columns:
    # Ignore all continuous variables, since they may unintentionally have a sum that is close to 0
    if col not in continuous_cols_lst and (sums.loc[col]==0 or sums.loc[col]==len(X_train)):
        lst_to_drop.append(col)

# Drop these columns from the overall set of features
background_imputed_tot = background_imputed_tot.drop(lst_to_drop, axis = 1)
print(background_imputed_tot.shape)


(4242, 507)
(4242, 5268)
(4242, 19729)
(4242, 19729)
(4242, 19233)
(4242, 19028)


In [10]:
# Save to pickle files
import pickle

with open('cleaned_data_dummy_vars.pickle', 'wb') as handle:
    pickle.dump(background_imputed_tot, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
X_train = background_imputed_tot.loc[X_train.index.values]
with open('X_train_dummy_vars.pickle', 'wb') as handle:
    pickle.dump(X_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
X_CV = background_imputed_tot.loc[X_CV.index.values]
with open('X_CV_dummy_vars.pickle', 'wb') as handle:
    pickle.dump(X_CV, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
x_test = background_imputed_tot.loc[test.index.values]
with open('x_test_dummy_vars.pickle', 'wb') as handle:
    pickle.dump(x_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
x_leaderboard = background_imputed_tot.loc[leaderboard.index.values]
with open('x_leaderboard_dummy_vars.pickle', 'wb') as handle:
    pickle.dump(x_leaderboard, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('y_train_dummy_vars.pickle', 'wb') as handle:
    pickle.dump(y_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('y_CV_dummy_vars.pickle', 'wb') as handle:
    pickle.dump(y_CV, handle, protocol=pickle.HIGHEST_PROTOCOL)